# Imports

In [191]:
%reset -f
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [192]:
import re

import pandas as pd

from controller import Controller

In [193]:
c = Controller('i01')

# Load data

In [194]:
filepath = fr'{c.get_path_data_original()}/asrs-aviation-reports-train.jsonl'
df_train = pd.read_json(path_or_buf=filepath, lines=True)
df_train['train_val_test_split'] = 'Train'
print(f'{df_train.shape=}')

filepath = fr'{c.get_path_data_original()}/asrs-aviation-reports-test.jsonl'
df_test = pd.read_json(path_or_buf=filepath, lines=True)
df_test['train_val_test_split'] = 'Test'
print(f'{df_test.shape=}')

filepath = fr'{c.get_path_data_original()}/asrs-aviation-reports-validation.jsonl'
df_validation = pd.read_json(path_or_buf=filepath, lines=True)
df_validation['train_val_test_split'] = 'Validation'
print(f'{df_validation.shape=}')

df_train.shape=(38655, 112)
df_test.shape=(4773, 112)
df_validation.shape=(4295, 112)


# Combine in one dataframe

In [195]:
df_train_val_test = pd.concat([df_train, df_validation, df_test], ignore_index=True)
print(f'{df_train_val_test.shape=}')

column = 'train_val_test_split'
pd.concat([
    df_train_val_test[column].value_counts(dropna=False),
    df_train_val_test[column].value_counts(dropna=False, normalize=True),
], axis=1, keys=['Total #', 'Total %'])

df_train_val_test.shape=(47723, 112)


,Total #,Total %
Train,38655,0.809987
Test,4773,0.100015
Validation,4295,0.089999


# Make all column names in capital case

In [196]:
def clean_column_names(name: str) -> str:
    # convert whitespace and punctuation to an underscore
    name = re.sub(r'[\s\W]+', '_', name)

    # make upper case
    name = name.upper()

    return name

In [197]:
print('############## Before ##############')
print(df_train_val_test.columns)

print()
print('############## After ##############')
print(df_train_val_test.columns.map(clean_column_names))

############## Before ##############
Index(['acn_num_ACN', 'Time_Date', 'Time.1_Local Time Of Day',
       'Place_Locale Reference', 'Place.1_State Reference',
       'Place.2_Relative Position.Angle.Radial',
       'Place.3_Relative Position.Distance.Nautical Miles',
       'Place.4_Altitude.AGL.Single Value',
       'Place.5_Altitude.MSL.Single Value', 'Environment_Flight Conditions',
       ...
       'Events.4_When Detected', 'Events.5_Result',
       'Assessments_Contributing Factors / Situations',
       'Assessments.1_Primary Problem', 'Report 1_Narrative',
       'Report 1.1_Callback', 'Report 2_Narrative', 'Report 2.1_Callback',
       'Report 1.2_Synopsis', 'train_val_test_split'],
      dtype='object', length=112)

############## After ##############
Index(['ACN_NUM_ACN', 'TIME_DATE', 'TIME_1_LOCAL_TIME_OF_DAY',
       'PLACE_LOCALE_REFERENCE', 'PLACE_1_STATE_REFERENCE',
       'PLACE_2_RELATIVE_POSITION_ANGLE_RADIAL',
       'PLACE_3_RELATIVE_POSITION_DISTANCE_NAUTICAL_MILE

In [198]:
df_train_val_test.columns = df_train_val_test.columns.map(clean_column_names)
df_train_val_test

,ACN_NUM_ACN,TIME_DATE,TIME_1_LOCAL_TIME_OF_DAY,PLACE_LOCALE_REFERENCE,PLACE_1_STATE_REFERENCE,PLACE_2_RELATIVE_POSITION_ANGLE_RADIAL,PLACE_3_RELATIVE_POSITION_DISTANCE_NAUTICAL_MILES,PLACE_4_ALTITUDE_AGL_SINGLE_VALUE,PLACE_5_ALTITUDE_MSL_SINGLE_VALUE,ENVIRONMENT_FLIGHT_CONDITIONS,...,EVENTS_4_WHEN_DETECTED,EVENTS_5_RESULT,ASSESSMENTS_CONTRIBUTING_FACTORS_SITUATIONS,ASSESSMENTS_1_PRIMARY_PROBLEM,REPORT_1_NARRATIVE,REPORT_1_1_CALLBACK,REPORT_2_NARRATIVE,REPORT_2_1_CALLBACK,REPORT_1_2_SYNOPSIS,TRAIN_VAL_TEST_SPLIT
0,1574675,201808,0601-1200,SNA.Airport,CA,,,,5000.0,,...,In-flight,Air Traffic Control Issued New Clearance; Flig...,Human Factors,Human Factors,SNA RNP-Z to Runway 20R. The FMC was properly ...,,We were cleared for the RNP RNAV Z 20R Approac...,,B737-700 flight crew reported failing to make ...,Train
1,1224894,201412,0601-1200,MSY.Airport,LA,,,1000.0,,VMC,...,In-flight,General None Reported / Taken,Human Factors,Human Factors,On base to final turn to runway 1 in MSY at ap...,,,,Captain reports sighting of a drone at 1;000 f...,Train
2,1134202,201312,1201-1800,ZZZ.ARTCC,US,,,,2600.0,IMC,...,In-flight,Air Traffic Control Provided Assistance; Air T...,Human Factors; Aircraft; Procedure; Weather,Aircraft,I climbed to my filed altitude of 5;000 FT; an...,,,,SR22 pilot became disoriented on approach in I...,Train
3,1222074,201411,1201-1800,CWA.Airport,WI,,20.0,,4000.0,,...,In-flight,Flight Crew Became Reoriented; General Mainten...,Aircraft,Aircraft,I had my pitot heat checked prior to winter an...,,,,BE58 pilot experiences pitot heat failure desc...,Train
4,1733019,202003,1801-2400,ZDV.ARTCC,CO,,,,32000.0,,...,In-flight,Air Traffic Control Issued New Clearance; Flig...,Airspace Structure; Weather,Weather,At 32000 ft. just north of PUB the aircraft ex...,,,,B737 First Officer reported unexpected moderat...,Train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47718,1341108,201603,0601-1200,ZZZ.Airport,US,,,,,IMC,...,In-flight,Air Traffic Control Provided Assistance; Fligh...,Company Policy; Human Factors,Human Factors,We did an originator out of ZZZ and had a main...,,[Report narrative contained no additional info...,,CRJ-900 flight crew reported being dispatched ...,Test
47719,1087474,201305,1201-1800,PHX.Airport,AZ,,,,8000.0,VMC,...,In-flight,Air Traffic Control Issued Advisory / Alert; F...,Aircraft; Human Factors; Procedure,Ambiguous,PHX takeoff Runway 25R flaps 5. CLEARANCE: MAX...,,There was an Airbus that departed before us an...,,CE750 flight crew departing PHX Runway 25R on ...,Test
47720,1756601,202008,,,,,,0.0,,,...,,General None Reported / Taken,Environment - Non Weather Related; Company Pol...,Company Policy,I was scheduled to complete the one day traini...,,,,Air carrier First Officer reported that re-qua...,Test
47721,1102938,201307,0601-1200,MEM.Airport,TN,,,,,VMC,...,In-flight,Air Traffic Control Issued New Clearance; Flig...,Airspace Structure; Chart Or Publication; Proc...,Human Factors,We departed Runway 36C in MEM via the GOETZ TW...,,,,On initial climb out via the GOETZ RNAV SID fr...,Test


# Clean event flag

In [199]:
column = 'EVENTS_5_RESULT'
pd.concat([
    df_train_val_test[column].value_counts(dropna=False),
    df_train_val_test[column].value_counts(dropna=False, normalize=True),
], axis=1, keys=['Total #', 'Total %'])

,Total #,Total %
General None Reported / Taken,7737,0.162123
Flight Crew Took Evasive Action,2776,0.058169
,2296,0.048111
General Maintenance Action,1736,0.036377
Air Traffic Control Issued New Clearance,1121,0.023490
...,...,...
Flight Crew FLC complied w / Automation / Advisory; Flight Crew Landed As Precaution; Flight Crew Returned To Departure Airport; Flight Crew Took Evasive Action,1,0.000021
Air Traffic Control Issued Advisory / Alert; Flight Crew Diverted; Flight Crew Landed As Precaution; General Declared Emergency; General Maintenance Action,1,0.000021
Flight Crew Became Reoriented; Flight Crew FLC Overrode Automation; Flight Crew Landed in Emergency Condition; Flight Crew Regained Aircraft Control,1,0.000021
Air Traffic Control Issued New Clearance; Air Traffic Control Issued Advisory / Alert; Flight Crew Took Evasive Action; Flight Crew Returned To Clearance; Flight Crew Regained Aircraft Control; Flight Crew Executed Go Around / Missed Approach,1,0.000021


In [204]:
def clean_even_outcome(outcome):
    # remove all "/" characters
    outcome = outcome.replace('/', '')
    
    # replace all whitespace characters with a single space
    outcome = re.sub(r'[\s]+', ' ', outcome)
    
    # split on the ";" character
    outcomes = outcome.split('; ')
    
    return outcomes

In [205]:
print('############## EVENTS_5_RESULT ##############')
print(df_train_val_test['EVENTS_5_RESULT'])

print()
print('############## EVENTS_5_RESULT_CLEAN ##############')
df_train_val_test['EVENTS_5_RESULT_CLEAN'] = df_train_val_test['EVENTS_5_RESULT'].map(clean_even_outcome)
print(df_train_val_test['EVENTS_5_RESULT_CLEAN'])

############## EVENTS_5_RESULT ##############
0        Air Traffic Control Issued New Clearance; Flig...
1                            General None Reported / Taken
2        Air Traffic Control Provided Assistance; Air T...
3        Flight Crew Became Reoriented; General Mainten...
4        Air Traffic Control Issued New Clearance; Flig...
                               ...                        
47718    Air Traffic Control Provided Assistance; Fligh...
47719    Air Traffic Control Issued Advisory / Alert; F...
47720                        General None Reported / Taken
47721    Air Traffic Control Issued New Clearance; Flig...
47722    General Flight Cancelled / Delayed; General Ma...
Name: EVENTS_5_RESULT, Length: 47723, dtype: object

############## EVENTS_5_RESULT_CLEAN ##############
0        [Air Traffic Control Issued New Clearance, Fli...
1                            [General None Reported Taken]
2        [Air Traffic Control Provided Assistance, Air ...
3        [Flight Crew B

In [206]:
df_train_val_test['EVENTS_5_RESULT_CLEAN'].apply(pd.Series).add_prefix('EVENTS_5_RESULT_CLEAN' + "_")

,EVENTS_5_RESULT_CLEAN_0,EVENTS_5_RESULT_CLEAN_1,EVENTS_5_RESULT_CLEAN_2,EVENTS_5_RESULT_CLEAN_3,EVENTS_5_RESULT_CLEAN_4,EVENTS_5_RESULT_CLEAN_5,EVENTS_5_RESULT_CLEAN_6,EVENTS_5_RESULT_CLEAN_7,EVENTS_5_RESULT_CLEAN_8,EVENTS_5_RESULT_CLEAN_9,EVENTS_5_RESULT_CLEAN_10
0,Air Traffic Control Issued New Clearance,Flight Crew Became Reoriented,Flight Crew Returned To Clearance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,General None Reported Taken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Air Traffic Control Provided Assistance,Air Traffic Control Issued New Clearance,Flight Crew Overcame Equipment Problem,Flight Crew Became Reoriented,Flight Crew Regained Aircraft Control,Flight Crew Requested ATC Assistance Clarifica...,NaN,NaN,NaN,NaN,NaN
3,Flight Crew Became Reoriented,General Maintenance Action,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Air Traffic Control Issued New Clearance,Flight Crew Requested ATC Assistance Clarifica...,Flight Crew Regained Aircraft Control,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
47718,Air Traffic Control Provided Assistance,Flight Crew Diverted,Flight Crew Landed in Emergency Condition,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47719,Air Traffic Control Issued Advisory Alert,Flight Crew Took Evasive Action,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47720,General None Reported Taken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47721,Air Traffic Control Issued New Clearance,Flight Crew Requested ATC Assistance Clarifica...,Flight Crew Returned To Clearance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
sorted(list(set(elem for sublist in df_train_val_test['EVENTS_5_RESULT_CLEAN'].values.tolist() for elem in sublist)))

['',
 'Air Traffic Control Issued Advisory Alert',
 'Air Traffic Control Issued New Clearance',
 'Air Traffic Control Provided Assistance',
 'Air Traffic Control Separated Traffic',
 'Aircraft Aircraft Damaged',
 'Aircraft Automated Return to Home (UAS)',
 'Aircraft Automation Overrode Flight Crew',
 'Aircraft Equipment Problem Dissipated',
 'Aircraft Lost Link (UAS)',
 'Aircraft Lost Unrecoverable (UAS)',
 'Flight Crew Became Reoriented',
 'Flight Crew Diverted',
 'Flight Crew Executed Go Around Missed Approach',
 'Flight Crew Exited Penetrated Airspace',
 'Flight Crew FLC Overrode Automation',
 'Flight Crew FLC complied w Automation Advisory',
 'Flight Crew Inflight Shutdown',
 'Flight Crew Landed As Precaution',
 'Flight Crew Landed in Emergency Condition',
 'Flight Crew Overcame Equipment Problem',
 'Flight Crew Overrode Automation',
 'Flight Crew Regained Aircraft Control',
 'Flight Crew Rejected Takeoff',
 'Flight Crew Requested ATC Assistance Clarification',
 'Flight Crew Return

In [228]:
'Aircraft Damaged' in df_train_val_test['EVENTS_5_RESULT'][0]

False

In [232]:
df_train_val_test[df_train_val_test['EVENTS_5_RESULT'].str.contains('Aircraft Damaged')][['EVENTS_5_RESULT', 'EVENTS_5_RESULT_CLEAN']]

,EVENTS_5_RESULT,EVENTS_5_RESULT_CLEAN
7,Aircraft Aircraft Damaged; Flight Crew Diverte...,"[Aircraft Aircraft Damaged, Flight Crew Divert..."
24,Aircraft Aircraft Damaged; Flight Crew Landed ...,"[Aircraft Aircraft Damaged, Flight Crew Landed..."
48,Aircraft Aircraft Damaged; General Flight Canc...,"[Aircraft Aircraft Damaged, General Flight Can..."
66,Aircraft Aircraft Damaged,[Aircraft Aircraft Damaged]
82,Air Traffic Control Provided Assistance; Aircr...,"[Air Traffic Control Provided Assistance, Airc..."
...,...,...
47556,Aircraft Aircraft Damaged,[Aircraft Aircraft Damaged]
47564,Aircraft Aircraft Damaged; Flight Crew Returne...,"[Aircraft Aircraft Damaged, Flight Crew Return..."
47640,Aircraft Aircraft Damaged; Flight Crew Overcam...,"[Aircraft Aircraft Damaged, Flight Crew Overca..."
47678,Aircraft Aircraft Damaged; General Maintenance...,"[Aircraft Aircraft Damaged, General Maintenanc..."


In [218]:
high_risk=['General Declared Emergency', 'General Physical Injury / Incapacitation', 'Flight Crew Inflight Shutdown', 'Air Traffic Control Separated Traffic', 'Aircraft Damaged', 'Aircraft Aircraft Damaged' ]
moderately_high_risk=['General Evacuated', 'Flight Crew Regained Aircraft Control', 'Air Traffic Control Issued Advisory / Alert', 'Flight Crew Landed in Emergency Condition', 'Flight Crew Landed In Emergency Condition']
medium_risk=['General Work Refused', 'Flight Crew Became Reoriented', 'Flight Crew Diverted', 'Flight Crew Executed Go Around / Missed Approach', 'Flight Crew Overcame Equipment Problem', 'Flight Crew Rejected Takeoff', 'Flight Crew Took Evasive Action', 'Air Traffic Control Issued New Clearance']
moderately_medium_risk=['General Maintenance Action', 'General Flight Cancelled / Delayed', 'General Release Refused / Aircraft Not Accepted', 'Flight Crew Overrode Automation', 'Flight Crew FLC Overrode Automation', 'Flight Crew Exited Penetrated Airspace', 'Flight Crew Requested ATC Assistance / Clarification', 'Flight Crew Landed As Precaution', 'Flight Crew Returned To Clearance', 'Flight Crew Returned To Departure Airport', 'Aircraft Automation Overrode Flight Crew']
low_risk=['General Police / Security Involved', 'Flight Crew Returned To Gate', 'Aircraft Equipment Problem Dissipated', 'Air Traffic Control Provided Assistance', 'General None Reported / Taken', 'Flight Crew FLC complied w / Automation / Advisory']

In [219]:
sorted(high_risk + moderately_high_risk + medium_risk + moderately_medium_risk + low_risk)

['Air Traffic Control Issued Advisory / Alert',
 'Air Traffic Control Issued New Clearance',
 'Air Traffic Control Provided Assistance',
 'Air Traffic Control Separated Traffic',
 'Aircraft Aircraft Damaged',
 'Aircraft Automation Overrode Flight Crew',
 'Aircraft Damaged',
 'Aircraft Equipment Problem Dissipated',
 'Flight Crew Became Reoriented',
 'Flight Crew Diverted',
 'Flight Crew Executed Go Around / Missed Approach',
 'Flight Crew Exited Penetrated Airspace',
 'Flight Crew FLC Overrode Automation',
 'Flight Crew FLC complied w / Automation / Advisory',
 'Flight Crew Inflight Shutdown',
 'Flight Crew Landed As Precaution',
 'Flight Crew Landed In Emergency Condition',
 'Flight Crew Landed in Emergency Condition',
 'Flight Crew Overcame Equipment Problem',
 'Flight Crew Overrode Automation',
 'Flight Crew Regained Aircraft Control',
 'Flight Crew Rejected Takeoff',
 'Flight Crew Requested ATC Assistance / Clarification',
 'Flight Crew Returned To Clearance',
 'Flight Crew Returne

In [ ]:
{
    'High risk': ['General Declared Emergency', 'General Physical Injury / Incapacitation', 'Flight Crew Inflight Shutdown']
}

# Saving to files

In [115]:
filepath = fr'{c.get_path_data_prepared()}/01_df_train_val_test.pkl'
df_train_val_test.to_pickle(filepath)